In [1]:
import os
import sys
currentdir = os.path.dirname(os.path.realpath('__file__'))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
print(currentdir, parentdir)

/Users/owl/Projects/normalization/notebooks /Users/owl/Projects/normalization


In [2]:
import yaml
import requests
from norm import *

In [3]:
content = requests.get('https://raw.githubusercontent.com/108michael/congress-legislators/master/legislators-historical.yaml').content
source = yaml.load(content)

In [4]:
df = pd.DataFrame(source)
# df['index'] = df.index
df_family = df[~df.family.isna()]
df_family.terms

11737    [{'type': 'sen', 'start': '2009-01-06', 'end':...
11739    [{'type': 'sen', 'start': '2009-01-06', 'end':...
11743    [{'type': 'sen', 'start': '1979-01-15', 'end':...
11744    [{'type': 'sen', 'start': '2003-01-07', 'end':...
11745    [{'type': 'sen', 'start': '1985-01-03', 'end':...
11746    [{'type': 'rep', 'start': '1999-01-06', 'end':...
11757    [{'type': 'rep', 'start': '1955-01-05', 'end':...
11765    [{'type': 'rep', 'start': '1999-01-06', 'end':...
Name: terms, dtype: object

In [ ]:
df_family.loc[:, 'family_obj'] = df.family.apply(lambda x: [1000, 2000, 3000])
df_family.family_obj

In [ ]:
df_family.loc[:, 'family_list'] = df.family.apply(lambda x: [['a', 'b'], ['a', 'b']])
df_family.family_list

In [30]:
def normalize3(dataframe):
    result = []
    for name, seri in dataframe.iteritems():
        inside = seri.apply(lambda x: type(x)).iloc[0]
        if inside == list:
            print('normalizing:: ', name, inside)
            temp = get_column_list(seri)
            temp = reset_index(temp)
            op = normalize3(temp)
            result.extend(op)
        elif inside == dict:
            print('normalizing:: ', name, inside)
            temp = get_column_dict(seri)
            temp = reset_index(temp)
            op = normalize3(temp)
            result.extend(op)
        else:
            temp = get_column_obj(seri)
            #print(temp.columns.values)
            result.append(temp)
            # print(len(result))
    print('exports :: ', len(result))
    return result

In [45]:
final = normalize(df_family.iloc[:4])

normalizing::  bio <class 'dict'>
normalizing::  family <class 'list'>
family <class 'dict'>
normalizing::  id <class 'dict'>
normalizing::  fec <class 'list'>
fec <class 'str'>


ValueError: can not merge DataFrame with instance of type <class 'str'>

In [55]:
final = normalize3(df_family.iloc[:4])

normalizing::  bio <class 'dict'>
exports ::  3
normalizing::  family <class 'list'>
family <class 'dict'>
exports ::  2
normalizing::  id <class 'dict'>
normalizing::  fec <class 'list'>
fec <class 'str'>
exports ::  1
exports ::  14
normalizing::  name <class 'dict'>
exports ::  3
normalizing::  terms <class 'list'>
terms <class 'dict'>
exports ::  14
exports ::  38


In [78]:
#len([print(i, v.columns.values) for i, v in enumerate(final)])
final[8]['index']
#final[2]['index']
final[37]

,index,url
0,11737,http://www.begich.senate.gov
1,11739,http://www.hagan.senate.gov
2,11743,NaN
3,11743,NaN
4,11743,NaN
5,11743,NaN
6,11743,http://levin.senate.gov/
7,11743,http://www.levin.senate.gov
8,11744,http://pryor.senate.gov
9,11744,http://www.pryor.senate.gov


In [86]:
reduce(lambda x, y: (pd.merge(x, y, on="index")), final)

KeyboardInterrupt: 

In [79]:
df_family.iloc[:2]

,bio,family,id,leadership_roles,name,other_names,terms
11737,"{'birthday': '1962-03-30', 'gender': 'M'}","[{'name': 'Nicholas J. Begich', 'relation': 's...","{'bioguide': 'B001265', 'thomas': '01898', 'li...",NaN,"{'first': 'Mark', 'last': 'Begich', 'official_...",NaN,"[{'type': 'sen', 'start': '2009-01-06', 'end':..."
11739,"{'birthday': '1953-05-26', 'gender': 'F'}","[{'name': 'Lawton M. Chiles Jr.', 'relation': ...","{'bioguide': 'H001049', 'thomas': '01902', 'li...",NaN,"{'first': 'Kay', 'last': 'Hagan', 'official_fu...",NaN,"[{'type': 'sen', 'start': '2009-01-06', 'end':..."
